## Rubric

Instructions: DELETE this cell before you submit via a `git push` to your repo before deadline. This cell is for your reference only and is not needed in your report. 

Scoring: Out of 10 points

- Each Developing  => -2 pts
- Each Unsatisfactory/Missing => -4 pts
  - until the score is 

If students address the detailed feedback in a future checkpoint they will earn these points back


|                  | Unsatisfactory                                                                                                                                                                                                    | Developing                                                                                                                                                                                              | Proficient                                     | Excellent                                                                                                                              |
|------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|
| Data relevance   | Did not have data relevant to their question. Or the datasets don't work together because there is no way to line them up against each other. If there are multiple datasets, most of them have this trouble | Data was only tangentially relevant to the question or a bad proxy for the question. If there are multiple datasets, some of them may be irrelevant or can't be easily combined.                       | All data sources are relevant to the question. | Multiple data sources for each aspect of the project. It's clear how the data supports the needs of the project.                         |
| Data description | Dataset or its cleaning procedures are not described. If there are multiple datasets, most have this trouble                                                                                              | Data was not fully described. If there are multiple datasets, some of them are not fully described                                                                                                      | Data was fully described                       | The details of the data descriptions and perhaps some very basic EDA also make it clear how the data supports the needs of the project. |
| Data wrangling   | Did not obtain data. They did not clean/tidy the data they obtained.  If there are multiple datasets, most have this trouble                                                                                 | Data was partially cleaned or tidied. Perhaps you struggled to verify that the data was clean because they did not present it well. If there are multiple datasets, some have this trouble | The data is cleaned and tidied.                | The data is spotless and they used tools to visualize the data cleanliness and you were convinced at first glance                      |


# COGS 108 - Data Checkpoint

## Authors

Instructions: REPLACE the contents of this cell with your team list and their contributions. Note that this will change over the course of the checkpoints

This is a modified [CRediT taxonomy of contributions](https://credit.niso.org). For each group member please list how they contributed to this project using these terms:
> Analysis, Background research, Conceptualization, Data curation, Experimental investigation, Methodology, Project administration, Software, Visualization, Writing – original draft, Writing – review & editing

Example team list and credits:
- Alice Anderson: Conceptualization, Data curation, Methodology, Writing - original draft
- Bob Barker:  Analysis, Software, Visualization
- Charlie Chang: Project administration, Software, Writing - review & editing
- Dani Delgado: Analysis, Background research, Visualization, Writing - original draft

## Research Question

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback



## Background and Prior Work

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Hypothesis


Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Data

### Data overview

Instructions: REPLACE the contents of this cell with descriptions of your actual datasets.

For each dataset include the following information
- Dataset #1
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
  - Description of the variables most relevant to this project
  - Descriptions of any shortcomings this dataset has with repsect to the project
- Dataset #2 (if you have more than one!)
  - same as above
- etc

Each dataset deserves either a set of bullet points as above or a few sentences if you prefer that method.

If you plan to use multiple datasets, add a few sentences about how you plan to combine these datasets.

In [1]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

In [18]:
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory
# if the data hasn't updated you don't need to do this again!

# if you don't already have these packages (you should!) uncomment this line
# %pip install requests tqdm

import sys
sys.path.append('./modules') # this tells python where to look for modules to import

import get_data # this is where we get the function we need to download data

# replace the urls and filenames in this list with your actual datafiles
# yes you can use Google drive share links or whatever
# format is a list of dictionaries; 
# each dict has keys of 
#   'url' where the resource is located
#   'filename' for the local filename where it will be stored 
datafiles = [
    { 'url': 'https://raw.githubusercontent.com/COGS108/Group084_WI26/cef6fe107a44418098ad1f67f63c8df132ba643f/data/00-raw/2018-2024%20Major%20Table.csv', 'filename':'2018-2024 Major Table.csv'},
    { 'url': 'https://raw.githubusercontent.com/COGS108/Group084_WI26/cef6fe107a44418098ad1f67f63c8df132ba643f/data/00-raw/UC%20alumni%20at%20work.csv', 'filename':'UC alumni at work.csv'}
]

get_data.get_raw(datafiles,destination_directory='data/00-raw/')

Overall Download Progress:  50%|█████     | 1/2 [00:00<00:00,  6.86it/s]          

Successfully downloaded: 2018-2024 Major Table.csv



Overall Download Progress: 100%|██████████| 2/2 [00:00<00:00,  6.87it/s]      

Successfully downloaded: UC alumni at work.csv


## Major Dataset: Outlier & Suspicious Entry Flags

In [23]:
df_major_flag = df_major.copy()
rules = []
rules.append(("bad_year", ~df_major_flag["year"].isin([2018, 2020, 2022, 2024])))
if "admit_rate" in df_major_flag.columns:
    rules.append(("bad_admit_rate", (df_major_flag["admit_rate"] < 0) | (df_major_flag["admit_rate"] > 1)))
if "yield_rate" in df_major_flag.columns:
    rules.append(("bad_yield_rate", (df_major_flag["yield_rate"] < 0) | (df_major_flag["yield_rate"] > 1)))

for c in ["applicants", "admits", "enrollees"]:
    if c in df_major_flag.columns:
        rules.append((f"neg_{c}", df_major_flag[c] < 0))

if set(["applicants","admits"]).issubset(df_major_flag.columns):
    rules.append(("admits_gt_applicants", df_major_flag["admits"] > df_major_flag["applicants"]))
if set(["enrollees","admits"]).issubset(df_major_flag.columns):
    rules.append(("enrollees_gt_admits", df_major_flag["enrollees"] > df_major_flag["admits"]))
if "gpa_mid" in df_major_flag.columns:
    rules.append(("bad_gpa_mid", (df_major_flag["gpa_mid"] < 0) | (df_major_flag["gpa_mid"] > 4.5)))

def flag_iqr_outliers(s, k=1.5):
    s = s.dropna()
    if len(s) < 6:
        return pd.Series(False, index=s.index) 
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo, hi = q1 - k * iqr, q3 + k * iqr
    return (s < lo) | (s > hi)

iqr_cols = [c for c in ["applicants","admits","enrollees","gpa_mid","admit_rate","yield_rate"] if c in df_major_flag.columns]
for col in iqr_cols:
    mask = (
        df_major_flag.groupby("year", group_keys=False)[col]
        .apply(lambda s: flag_iqr_outliers(s, k=1.5))
    )
    df_major_flag[f"out_{col}_iqr"] = False
    df_major_flag.loc[mask.index, f"out_{col}_iqr"] = mask

df_major_flag["suspicious_reasons"] = ""
for name, cond in rules:
    df_major_flag.loc[cond.fillna(False), "suspicious_reasons"] += f"{name};"

for col in iqr_cols:
    flag_col = f"out_{col}_iqr"
    df_major_flag.loc[df_major_flag[flag_col] == True, "suspicious_reasons"] += f"{flag_col};"

df_major_flag["is_suspicious"] = df_major_flag["suspicious_reasons"].str.len() > 0
print("Suspicious rows (major):", df_major_flag["is_suspicious"].sum())
display(df_major_flag[df_major_flag["is_suspicious"]].head(30))

Suspicious rows (major): 9


,year,major,applicants,admits,enrollees,admit_rate,yield_rate,out_applicants_iqr,out_admits_iqr,out_enrollees_iqr,out_admit_rate_iqr,out_yield_rate_iqr,suspicious_reasons,is_suspicious
6,2024,Cognitive Science,358,294,140,0.82,0.48,False,True,True,False,False,out_admits_iqr;out_enrollees_iqr;,True
21,2022,Cognitive Science,313,257,107,0.82,0.42,False,False,True,False,False,out_enrollees_iqr;,True
36,2020,Cognitive Science,278,195,86,0.70,0.44,False,False,True,False,False,out_enrollees_iqr;,True
44,2020,Mechanical Engineering,540,149,40,0.28,0.27,True,False,False,False,False,out_applicants_iqr;,True
49,2018,Electrical Engineering - BS,435,221,69,0.51,0.31,True,False,True,False,False,out_applicants_iqr;out_enrollees_iqr;,True
51,2018,Cognitive Science,254,173,73,0.68,0.42,False,False,True,False,False,out_enrollees_iqr;,True
54,2018,Literature/Writing,101,57,14,0.56,0.25,False,False,True,False,False,out_enrollees_iqr;,True
58,2018,"Ecology, Behavior & Evolution",66,22,4,0.33,0.18,False,False,True,False,False,out_enrollees_iqr;,True
59,2018,Mechanical Engineering,560,137,76,0.24,0.55,True,False,True,False,False,out_applicants_iqr;out_enrollees_iqr;,True


## Earnings Dataset: Outlier & Suspicious Entry Flags

In [24]:
df_earn_flag = df_earn.copy()
rules = []
rules.append(("bad_year", ~df_earn_flag["year"].isin([2018, 2020, 2022, 2024])))
for c in ["earn_p25","earn_med","earn_p75"]:
    if c in df_earn_flag.columns:
        rules.append((f"neg_{c}", df_earn_flag[c] < 0))
        rules.append((f"too_big_{c}", df_earn_flag[c] > 1_000_000))
if set(["earn_p25","earn_med"]).issubset(df_earn_flag.columns):
    rules.append(("p25_gt_med", df_earn_flag["earn_p25"] > df_earn_flag["earn_med"]))
if set(["earn_med","earn_p75"]).issubset(df_earn_flag.columns):
    rules.append(("med_gt_p75", df_earn_flag["earn_med"] > df_earn_flag["earn_p75"]))

for c in ["earners_count","records_count"]:
    if c in df_earn_flag.columns:
        rules.append((f"neg_{c}", df_earn_flag[c] < 0))

def flag_iqr_outliers(s, k=1.5):
    s = s.dropna()
    if len(s) < 6:
        return pd.Series(False, index=s.index)
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo, hi = q1 - k * iqr, q3 + k * iqr
    return (s < lo) | (s > hi)

iqr_cols = [c for c in ["earn_p25","earn_med","earn_p75","earners_count"] if c in df_earn_flag.columns]
for col in iqr_cols:
    mask = (
        df_earn_flag.groupby("year", group_keys=False)[col]
        .apply(lambda s: flag_iqr_outliers(s, k=1.5))
    )
    df_earn_flag[f"out_{col}_iqr"] = False
    df_earn_flag.loc[mask.index, f"out_{col}_iqr"] = mask

df_earn_flag["suspicious_reasons"] = ""
for name, cond in rules:
    df_earn_flag.loc[cond.fillna(False), "suspicious_reasons"] += f"{name};"
for col in iqr_cols:
    flag_col = f"out_{col}_iqr"
    df_earn_flag.loc[df_earn_flag[flag_col] == True, "suspicious_reasons"] += f"{flag_col};"

df_earn_flag["is_suspicious"] = df_earn_flag["suspicious_reasons"].str.len() > 0
print("Suspicious rows (earnings):", df_earn_flag["is_suspicious"].sum())
display(df_earn_flag[df_earn_flag["is_suspicious"]].head(30))

Suspicious rows (earnings): 6


,year,major,earn_med,earn_p25,earn_p75,earners_count,records_count,out_earn_p25_iqr,out_earn_med_iqr,out_earn_p75_iqr,out_earners_count_iqr,suspicious_reasons,is_suspicious
2,2024,Computer Engineering,104844.0,76024.0,160505.0,156,156,False,False,True,True,out_earn_p75_iqr;out_earners_count_iqr;,True
4,2024,Electrical Engineering - BS,92359.0,79534.0,113449.0,172,172,False,False,False,True,out_earners_count_iqr;,True
17,2022,Computer Engineering,116188.0,78670.0,156585.0,127,127,False,False,True,False,out_earn_p75_iqr;,True
21,2022,Cognitive Science,59401.0,41686.0,81054.0,254,254,False,False,False,True,out_earners_count_iqr;,True
32,2020,Computer Engineering,121057.0,95474.0,160247.0,174,174,False,False,True,False,out_earn_p75_iqr;,True
47,2018,Computer Engineering,109959.0,88017.0,145813.0,174,174,False,False,True,False,out_earn_p75_iqr;,True


### Major Dataset: UC Major Selectivity & GPA (2018–2024)

Instructions: 
1. Change the header from Dataset #1 to something more descriptive of the dataset
2. Write a few paragraphs about this dataset. Make sure to cover
   1. Describe the important metrics, what units they are in, and giv some sense of what they mean.  For example "Fasting blood glucose in units of mg glucose per deciliter of blood.  Normal values for healthy individuals range from 70 to 100 mg/dL.  Values 100-125 are prediabetic and values >125mg/dL indicate diabetes. Values <70 indicate hypoglycemia. Fasting idicates the patient hasn't eaten in the last 8 hours.  If blood glucose is >250 or <50 at any time (regardless of the time of last meal) the patient's life may be in immediate danger"
   2. If there are any major concerns with the dataset, describe them. For example "Dataset is composed of people who are serious enough about eating healthy that they voluntarily downloaded an app dedicated to tracking their eating patterns. This sample is likely biased because of that self-selection. These people own smartphones and may be healthier and may have more disposable income than the average person.  Those who voluntarily log conscientiously and for long amounts of time are also likely even more interested in health than those who download the app and only log a bit before getting tired of it"
3. Use the cell below to 
    1. load the dataset 
    2. make the dataset tidy or demonstrate that it was already tidy
    3. demonstrate the size of the dataset
    4. find out how much data is missing, where its missing, and if its missing at random or seems to have any systematic relationships in its missingness
    5. find and flag any outliers or suspicious entries
    6. clean the data or demonstrate that it was already clean.  You may choose how to deal with missingness (dropna of fillna... how='any' or 'all') and you should justify your choice in some way
    7. You will load raw data from `data/00-raw/`, you will (optionally) write intermediate stages of your work to `data/01-interim` and you will write the final fully wrangled version of your data to `data/02-processed`
4. Optionally you can also show some summary statistics for variables that you think are important to the project
5. Feel free to add more cells here if that's helpful for you


In [25]:
import os, re
import numpy as np
import pandas as pd

# ---- paths (repo template folders) ----
RAW_DIR = "data/00-raw"
PROC_DIR = "data/02-processed"
os.makedirs(PROC_DIR, exist_ok=True)

major_path = os.path.join(RAW_DIR, "2018-2024 Major Table.csv")

df_major_raw = pd.read_csv(
    major_path,
    encoding="utf-8-sig",
    sep=",",
    engine="python",
)

print("Major raw shape:", df_major_raw.shape)
display(df_major_raw.head())

def to_numeric_percent(x):
    """'75%' -> 0.75, '0.75'->0.75, NaN stays NaN"""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == "":
        return np.nan
    s = s.replace("%", "")
    try:
        v = float(s)
    except:
        return np.nan
    # if user stored 75 instead of 0.75
    if v > 1.0:
        v = v / 100.0
    return v

def parse_gpa_range(x):
    """'3.41 - 3.96' -> (3.41, 3.96); returns (nan,nan) if fails"""
    if pd.isna(x):
        return (np.nan, np.nan)
    s = str(x)
    # normalize dash variants
    s = s.replace("–", "-").replace("—", "-")
    # extract two numbers
    nums = re.findall(r"\d+\.\d+|\d+", s)
    if len(nums) >= 2:
        lo = float(nums[0]); hi = float(nums[1])
        if lo > hi:
            lo, hi = hi, lo
        return (lo, hi)
    return (np.nan, np.nan)

def standardize_major_name(x):
    """basic cleanup; keep it conservative"""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    return s
rename_map = {
    "Year": "year",
    "Major": "major",
    "Applicants": "applicants",
    "Admits": "admits",
    "Enrollees": "enrollees",
    "Admit GPA": "admit_gpa_range",
    "Enrollee GPA": "enrollee_gpa_range",
    "Admit rate": "admit_rate",
    "Yield rate": "yield_rate",
}
df_major = df_major_raw.rename(columns=rename_map).copy()

keep_cols = [c for c in rename_map.values() if c in df_major.columns]
df_major = df_major[keep_cols].copy()
df_major["year"] = pd.to_numeric(df_major["year"], errors="coerce").astype("Int64")
df_major["major"] = df_major["major"].apply(standardize_major_name)
lo, hi = zip(*df_major["admit_gpa_range"].apply(parse_gpa_range)) if "admit_gpa_range" in df_major.columns else ([], [])
if len(lo) > 0:
    df_major["gpa_low"] = lo
    df_major["gpa_high"] = hi
    df_major["gpa_mid"] = (df_major["gpa_low"] + df_major["gpa_high"]) / 2
    df_major["gpa_width"] = df_major["gpa_high"] - df_major["gpa_low"]

if "admit_rate" in df_major.columns:
    df_major["admit_rate"] = df_major["admit_rate"].apply(to_numeric_percent)
if "yield_rate" in df_major.columns:
    df_major["yield_rate"] = df_major["yield_rate"].apply(to_numeric_percent)

for c in ["applicants", "admits", "enrollees"]:
    if c in df_major.columns:
        df_major[c] = pd.to_numeric(df_major[c], errors="coerce")

YEARS = [2018, 2020, 2022, 2024]
df_major = df_major[df_major["year"].isin(YEARS)].copy()

df_major = df_major.dropna(subset=["year", "major"]).copy()

print("Major cleaned shape:", df_major.shape)
display(df_major.head(10))

miss = df_major.isna().mean().sort_values(ascending=False)
print("Major missingness (fraction):")
display(miss)

major_out = os.path.join(PROC_DIR, "major_processed.csv")
df_major.to_csv(major_out, index=False, encoding="utf-8-sig")
print("Saved:", major_out)

Major raw shape: (60, 9)


,Year,Major,Applicants,Admits,Enrollees,Admit GPA range,Enrollee GPA range,Admit rate,Yield rate
0,2024,History,284,212,24,3.41 - 3.96,3.16 - 3.72,75%,11%
1,2024,Visual Arts - Media,263,176,63,3.41 - 3.88,3.44 - 3.82,67%,36%
2,2024,Computer Engineering,455,137,61,3.70 - 3.97,3.72 - 3.93,30%,45%
3,2024,Chemistry,190,89,28,3.41 - 3.98,3.40 - 3.69,47%,31%
4,2024,Electrical Engineering - BS,412,161,68,3.51 - 3.95,3.40 - 3.82,39%,42%


Major cleaned shape: (60, 7)


,year,major,applicants,admits,enrollees,admit_rate,yield_rate
0,2024,History,284,212,24,0.75,0.11
1,2024,Visual Arts - Media,263,176,63,0.67,0.36
2,2024,Computer Engineering,455,137,61,0.30,0.45
3,2024,Chemistry,190,89,28,0.47,0.31
4,2024,Electrical Engineering - BS,412,161,68,0.39,0.42
5,2024,Bioengineering,95,41,14,0.43,0.34
6,2024,Cognitive Science,358,294,140,0.82,0.48
7,2024,Public Health,217,128,56,0.59,0.44
8,2024,Applied Mathematics,194,143,46,0.74,0.32
9,2024,Literature/Writing,153,90,13,0.59,0.14


Major missingness (fraction):


year          0.0
major         0.0
applicants    0.0
admits        0.0
enrollees     0.0
admit_rate    0.0
yield_rate    0.0
dtype: float64

Saved: data/02-processed/major_processed.csv


### Earnings Dataset: UC Alumni Earnings by Major (2018–2024) 

See instructions above for Dataset #1.  Feel free to keep adding as many more datasets as you need.  Put each new dataset in its own section just like these. 

Lastly if you do have multiple datasets, add another section where you demonstrate how you will join, align, cross-reference or whatever to combine data from the different datasets

Please note that you can always keep adding more datasets in the future if these datasets you turn in for the checkpoint aren't sufficient.  The goal here is demonstrate that you can obtain and wrangle data.  You are not tied down to only use what you turn in right now.

In [26]:
import os, re
import numpy as np
import pandas as pd

RAW_DIR = "data/00-raw"
PROC_DIR = "data/02-processed"
os.makedirs(PROC_DIR, exist_ok=True)
earn_path = os.path.join(RAW_DIR, "UC alumni at work.csv")

df_earn_raw = pd.read_csv(
    earn_path,
    encoding="utf-8-sig",
    sep=",",
    engine="python",
)
print("Earnings raw shape:", df_earn_raw.shape)
display(df_earn_raw.head())

def standardize_major_name(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    return s

def to_numeric_money(x):
    """handles '120,548' or '$120,548' etc."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == "":
        return np.nan
    s = s.replace("$", "").replace(",", "")
    try:
        return float(s)
    except:
        return np.nan

rename_map = {
    "Year": "year",
    "Major": "major",
    "Median Annual Earnings": "earn_med",
    "Percentile (25) of Annual Earnings": "earn_p25",
    "Percentile (75) of Annual Earnings": "earn_p75",
    "Earners Count": "earners_count",
    "Count of Number of Records": "records_count",
}
df_earn = df_earn_raw.rename(columns=rename_map).copy()

keep_cols = [c for c in rename_map.values() if c in df_earn.columns]
df_earn = df_earn[keep_cols].copy()
def normalize_year(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    m = re.match(r"^\s*(\d{4})\s*-\s*(\d{4})\s*$", s)
    if m:
        return int(m.group(2))
    try:
        return int(float(s))
    except:
        return np.nan

df_earn["year"] = df_earn["year"].apply(normalize_year).astype("Int64")
df_earn["major"] = df_earn["major"].apply(standardize_major_name)

for c in ["earn_med", "earn_p25", "earn_p75"]:
    if c in df_earn.columns:
        df_earn[c] = df_earn[c].apply(to_numeric_money)
for c in ["earners_count", "records_count"]:
    if c in df_earn.columns:
        df_earn[c] = pd.to_numeric(df_earn[c], errors="coerce")

YEARS = [2018, 2020, 2022, 2024]
df_earn = df_earn[df_earn["year"].isin(YEARS)].copy()

df_earn = df_earn.dropna(subset=["year", "major"]).copy()

print("Earnings cleaned shape:", df_earn.shape)
display(df_earn.head(10))

miss = df_earn.isna().mean().sort_values(ascending=False)
print("Earnings missingness (fraction):")
display(miss)

earn_out = os.path.join(PROC_DIR, "earnings_processed.csv")
df_earn.to_csv(earn_out, index=False, encoding="utf-8-sig")
print("Saved:", earn_out)

Earnings raw shape: (60, 7)


,Year,Major,Percentile (75) of Annual Earnings,Median Annual Earnings,Percentile (25) of Annual Earnings,Earners Count,Count of Number of Records
0,2024,History,49697,35933,21804,61,61
1,2024,Visual Arts - Media,49000,36057,20440,60,60
2,2024,Computer Engineering,160505,104844,76024,156,156
3,2024,Chemistry,71287,56269,48061,80,80
4,2024,Electrical Engineering - BS,113449,92359,79534,172,172


Earnings cleaned shape: (60, 7)


,year,major,earn_med,earn_p25,earn_p75,earners_count,records_count
0,2024,History,35933.0,21804.0,49697.0,61,61
1,2024,Visual Arts - Media,36057.0,20440.0,49000.0,60,60
2,2024,Computer Engineering,104844.0,76024.0,160505.0,156,156
3,2024,Chemistry,56269.0,48061.0,71287.0,80,80
4,2024,Electrical Engineering - BS,92359.0,79534.0,113449.0,172,172
5,2024,Bioengineering,61865.0,39206.0,83579.0,40,40
6,2024,Cognitive Science,44596.0,30211.0,67291.0,153,153
7,2024,Public Health,45419.0,29155.0,62040.0,62,62
8,2024,Applied Mathematics,56440.0,32556.0,72668.0,60,60
9,2024,Literature/Writing,38368.0,27740.0,50520.0,71,71


Earnings missingness (fraction):


year             0.0
major            0.0
earn_med         0.0
earn_p25         0.0
earn_p75         0.0
earners_count    0.0
records_count    0.0
dtype: float64

Saved: data/02-processed/earnings_processed.csv


## Ethics

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Team Expectations 

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Project Timeline Proposal

Instructions: Replace this with your timeline.  **PLEASE UPDATE your Timeline!** No battle plan survives contact with the enemy, so make sure we understand how your plans have changed.  Also if you have lost points on the previous checkpoint fix them